In [ ]:
# Remove the first 10 rows (0-9) which contain metadata from df_gdp_overall
df_gdp_fact_data = df_gdp_overall.iloc[10:].copy()
date_column = df_gdp_fact_data['Unnamed: 0']
indicator_columns = df_gdp_fact_data.columns[1:]
df_gdp_fact_normalized = df_gdp_fact_data.melt(
    id_vars=['Unnamed: 0'],
    value_vars=indicator_columns,
    var_name='IndicatorID',
    value_name='Value'
)
df_gdp_fact_normalized = df_gdp_fact_normalized.rename(columns={'Unnamed: 0': 'Date'})
display(df_gdp_fact_normalized.head())

In [ ]:
# Clean and convert types
df_gdp_fact_normalized['Date'] = pd.to_datetime(df_gdp_fact_normalized['Date'], errors='coerce')
df_gdp_fact_normalized['Value'] = pd.to_numeric(df_gdp_fact_normalized['Value'], errors='coerce')
df_gdp_fact_normalized['IndicatorID'] = df_gdp_fact_normalized['IndicatorID'].astype(str)
display(df_gdp_fact_normalized.info())

In [ ]:
# Recreate indicator mapping
df_gdp_fact_data = df_gdp_overall.iloc[10:].copy()
indicator_columns = df_gdp_fact_data.columns[1:]
df_gdp_fact_normalized = df_gdp_fact_data.melt(
    id_vars=['Unnamed: 0'],
    value_vars=indicator_columns,
    var_name='IndicatorName_original',
    value_name='Value'
)
df_gdp_fact_normalized = df_gdp_fact_normalized.rename(columns={'Unnamed: 0': 'Date'})
unique_indicator_names = df_gdp_fact_normalized['IndicatorName_original'].unique()
indicator_id_map = {name: i + 1 for i, name in enumerate(unique_indicator_names)}
df_gdp_fact_normalized['IndicatorID'] = df_gdp_fact_normalized['IndicatorName_original'].map(indicator_id_map)
df_gdp_fact_normalized = df_gdp_fact_normalized.rename(columns={'IndicatorName_original': 'Indicator Name'})
df_gdp_fact_normalized['Indicator Name'] = df_gdp_fact_normalized['Indicator Name'].str.replace(';', '', regex=False).str.strip()
df_gdp_fact_normalized = df_gdp_fact_normalized[['Date', 'IndicatorID', 'Indicator Name', 'Value']]
display(df_gdp_fact_normalized.head())

In [ ]:
# Extract metadata rows
df_metadata = df_gdp_overall.iloc[:10].copy()
df_metadata_transposed = df_metadata.transpose().reset_index(drop=True)
df_metadata_transposed.columns = df_metadata_transposed.iloc[0]
df_metadata_transposed = df_metadata_transposed[1:].copy()
df_metadata_transposed = df_metadata_transposed.rename(columns={df_metadata_transposed.columns[0]: 'Indicator Name'})
df_metadata_transposed['Indicator Name'] = df_metadata_transposed['Indicator Name'].str.replace(';', '', regex=False).str.strip()
unique_indicator_names_cleaned = df_metadata_transposed['Indicator Name'].unique()
indicator_name_to_id_map = {name: i + 1 for i, name in enumerate(unique_indicator_names_cleaned)}
df_metadata_transposed['IndicatorID'] = df_metadata_transposed['Indicator Name'].map(indicator_name_to_id_map)
dim_table = df_metadata_transposed[['IndicatorID', 'Indicator Name', 'Series Type', 'Data Type', 'Frequency', 'Collection Month', 'Series Start', 'Series End', 'No. Obs']].copy()
display(dim_table.head())

In [ ]:
# Export both tables
df_gdp_fact_normalized.to_csv('gdp_fact_table.csv', index=False)
dim_table.to_csv('gdp_dim_table.csv', index=False)
print('Exports complete.')